# Running Wflow using the ewatercycle system

### 1. Copy an example case
To run WFlow, we need a complete parameterset. The easiest way to obtain a valid model configuration is by copying it from https://github.com/openstreams/wflow/raw/master/examples/. We can use `ewatercycle.parametersetdb` to easily copy on of these example cases to a folder called `./wflow_example_case`.

In [ ]:
from ewatercycle.parametersetdb import build_from_urls

# Obtain an example case for testing the model
parameterset = build_from_urls(
    config_format='ini', config_url='https://github.com/openstreams/wflow/raw/master/examples/wflow_rhine_sbm/wflow_sbm.ini',
    datafiles_format='svn', datafiles_url='https://github.com/openstreams/wflow/trunk/examples/wflow_rhine_sbm',
)
parameterset.save_datafiles('./wflow_example_case')

### 2. Verify your configuration settings

If you are working on a curated environment like https://lab.ewatercycle.org/, the default configuration should cover 90% of all use cases. However, for the other 10% here is a brief explanation of the relevant configuration settings. 

The ewatercycle configuration file is typcially stored in `~/ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for WFLow are:

```Yaml
container_engine: docker # or singularity
singularity_images.wflow: 'ewatercycle-wflow-grpc4bmi.sif'
docker_images.wflow: 'ewatercycle/wflow-grpc4bmi:latest'
```

In order for this to work, you need to make sure that the actual docker or singularity images are available on the system. This can be done with:

- Docker: `docker pull ewatercycle/wflow-grpc4bmi:latest`
- Singularity: `singularity build --disable-cache ewatercycle-wflow-grpc4bmi.sif docker://ewatercycle/wflow-grpc4bmi:latest`

### 3. Setting up the model

In [1]:
from ewatercycle.models import Wflow

In [2]:
model = Wflow()

In [3]:
model.parameters

First run setup with a valid .ini file.


In [4]:
cfg_dir, cfg_file = model.setup(cfg_dir='wflow_example_case/', cfg_file='./wflow_example_case/wflow_sbm.ini')

Working directory created: wflow_20210413_175541
Created /home/peter/ewatercycle/ewatercycle/examples/wflow_20210413_175541/wflow_ewatercycle.ini.
Started wflow container with working directory /home/peter/ewatercycle/ewatercycle/examples/wflow_20210413_175541


In [5]:
cfg_dir, cfg_file

('/home/peter/ewatercycle/ewatercycle/examples/wflow_20210413_175541',
 'wflow_ewatercycle.ini')

The `config_file` that is returned by `setup(...)` can be used to initialize the model:

In [6]:
# Print a list of currently used parameters
model.parameters

inputmapstacks
    Precipitation: /inmaps/P
    EvapoTranspiration: /inmaps/PET
    Temperature: /inmaps/TEMP
    Inflow: /inmaps/IF
run
    starttime: 1995-01-31 00:00:00 GMT
    endtime: 1995-02-28 00:00:00 GMT
    runinfo: None
    timestepsecs: 86400
    reinit: 0
model
    modeltype: sbm
    AnnualDischarge: 2290
    Alpha: 120
    ModelSnow: 0
    ScalarInput: 0
    InterpolationMethod: inv
    WIMaxScale: 0.8
    Tslice: 1
    UpdMaxDist: 300000.0
    origTopogLateral: 1
    configfile: wflow_sbm.ini
    reinit: 0
    fewsrun: 0
    OverWriteInit: 0
    updating: 0
    updateFile: no_set
    lateralmethod: 1
    sCatch: 0
    intbl: intbl
    timestepsecs: 86400
    MaxUpdMult: 1.3
    MinUpdMult: 0.7
    UpFrac: 0.8
    waterdem: 0
    reInfilt: 0
    MassWasting: 0
    wflow_subcatch: staticmaps/wflow_subcatch.map
    wflow_dem: staticmaps/wflow_dem.map
    wflow_ldd: staticmaps/wflow_ldd.map
    wflow_river: staticmaps/wflow_river.map
    wflow_riverlength: staticmaps/wflow_r

### 4. Running the model

In [ ]:
model.initialize(cfg_file)

In [ ]:
model.get_value("SurfaceRunoff")

In [ ]:
model.get_value_as_xarray("SurfaceRunoff")

In [ ]:
while model.time < model.end_time:
    print(model.time)
    model.update()

In [ ]:
da = model.get_value_as_xarray("SurfaceRunoff")
da.plot()

In [ ]:
all(model.get_value("SurfaceRunoff")==0)

In [ ]:
da

In [ ]:
for i in range(100):
    model.bmi.update()

In [ ]:
model.finalize()

In [ ]:
model.input_var_names